In [1]:
# General
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Torch
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset
from torchvision.datasets import ImageFolder
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
from datetime import datetime

# Pillow
from PIL import Image
import pickle

In [3]:
device = torch.device('cpu')
export_file = 'final_model.pth'

from final_model import Conv2D, MaxPool, Dense, CNNModel
model = CNNModel()

model.load_state_dict(torch.load(export_file,map_location=device))
model.eval()

torch.Size([4096, 1024])
torch.Size([1024])
torch.Size([1024, 7])
torch.Size([7])


CNNModel(
  (c1): Conv2D()
  (m1): MaxPool()
  (dropout): Dropout(p=0.25, inplace=False)
  (c2): Conv2D()
  (m2): MaxPool()
  (c3): Conv2D()
  (m3): MaxPool()
  (c4): Conv2D()
  (m4): MaxPool()
  (c5): Conv2D()
  (m5): MaxPool()
  (d1): Dense()
  (d2): Dense()
)

In [22]:
# import pickle
# from model import CNNModel
# from model import Conv2D, MaxPool, Dense
# # Load the PyTorch model from a .pkl file
# with open('model.pkl', 'rb') as f:
#     model = pickle.load(f)

# model.eval()

In [4]:
idx_to_class = {
    0: 'Bacterial_spot',
    1: 'Early_blight',
    2: 'Healthy',
    3: 'Late_blight',
    4: 'Mosiac_virus',
    5: 'Septoria_leaf_spot',
    6: 'YellowLeaf_curl'
 }

In [5]:

def predict_disease(img):
    image = Image.open(img).convert('RGB')

    # Apply transformations
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = transform(image)

    # Pass through the model
    model.eval()
    with torch.no_grad():
        #image = image # Move input tensor to the GPU
        output = model(image.unsqueeze(0))  # Add batch dimension
    predicted_probs = torch.softmax(output, dim=1)
    predicted_class = torch.argmax(output, dim=1).item()
    actual_class = idx_to_class[predicted_class]
    confidence = predicted_probs[0][predicted_class].item()

    # Print the predicted class and confidence
    print('Predicted class:', actual_class)
    print('Confidence:', confidence)

    return actual_class,confidence

In [6]:
img = "C:/Users/Saurab/Desktop/Tomato Leaf disease detection/early_blight21.jpg"

predict_disease(img)

Predicted class: Early_blight
Confidence: 0.9951351284980774


('Early_blight', 0.9951351284980774)

In [7]:
data_path = "C:/Users/Saurab/Desktop/Plant Disease Final/Dataset/Plant"
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = datasets.ImageFolder(root=data_path, transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
# Get the first batch of data
inputs, labels = next(iter(dataloader))

# Get the shape of the images
image_shape = inputs[0].shape

# Print the number of images and their shape
print(f"Number of images: {len(dataloader.dataset)}")
print(f"Image shape: {image_shape}")

Number of images: 14315
Image shape: torch.Size([3, 256, 256])


In [8]:
indices = list(range(len(dataset)))
split = int(np.floor(0.90 * len(dataset)))  # train_size
validation = int(np.floor(0.70 * split))   # validation
np.random.shuffle(indices)

print(f"length of train size :{validation}")
print(f"length of validation size :{split - validation}")
print(f"length of test size :{len(dataset)-validation}")

train_indices, validation_indices, test_indices = (
    indices[:validation],
    indices[validation:split],
    indices[split:],
)

length of train size :9018
length of validation size :3865
length of test size :5297


In [9]:
train_sampler = SubsetRandomSampler(train_indices)
validation_sampler = SubsetRandomSampler(validation_indices)
test_sampler = SubsetRandomSampler(test_indices)

In [10]:
indices = list(range(len(dataset)))
split = int(np.floor(0.90 * len(dataset)))  # train_size
validation = int(np.floor(0.70 * split))   # validation
np.random.shuffle(indices)

print(f"length of train size :{validation}")
print(f"length of validation size :{split - validation}")
print(f"length of test size :{len(dataset)-validation}")

length of train size :9018
length of validation size :3865
length of test size :5297


In [11]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, sampler=train_sampler
)
test_loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, sampler=test_sampler
)
validation_loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, sampler=validation_sampler
)

In [12]:
def accuracy(loader):
    n_correct = 0
    n_total = 0

    for inputs, targets in loader:
        inputs, targets = inputs.to(device), targets.to(device)

        outputs = model(inputs)

        _, predictions = torch.max(outputs, 1)

        n_correct += (predictions == targets).sum().item()
        n_total += targets.shape[0]

    acc = n_correct / n_total
    return acc

train_acc = accuracy(train_loader)
test_acc = accuracy(test_loader)
validation_acc = accuracy(validation_loader)

print(
    f"Train Accuracy : {train_acc}\nTest Accuracy : {test_acc}\nValidation Accuracy : {validation_acc}"
)

Train Accuracy : 0.980261698824573
Test Accuracy : 0.9776536312849162
Validation Accuracy : 0.9793014230271668
